In [2]:
from sqlalchemy import create_engine
import pandas as pd
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup as bs

DEFAULT_TIMEOUT = 2
MAX_ATTEMPT = 5


class TimeoutHTTPAdapter(HTTPAdapter):
    def __init__(self, *args, **kwargs):
        self.timeout = DEFAULT_TIMEOUT
        if "timeout" in kwargs:
            self.timeout = kwargs["timeout"]
            del kwargs["timeout"]
        super().__init__(*args, **kwargs)

    def send(self, request, **kwargs):
        timeout = kwargs.get("timeout")
        if timeout is None:
            kwargs["timeout"] = self.timeout
        return super().send(request, **kwargs)

assets = pd.read_csv('./Data/Base_Data.csv')
mktdata = pd.DataFrame()

def getdata(url, sess, attempt):
    website = None
    try:
        website = sess.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=5)
        
    except requests.exceptions.RequestException as e:
        print('except')
        if attempt < MAX_ATTEMPT:
            getdata(url, sess, attempt+1)
        else:   
            print(e)
            website = None

    except ConnectionError as e:
        print('except')
        if attempt < MAX_ATTEMPT:
            getdata(url, sess, attempt+1)
        else:
            print(e)
            website = None

    return website
    

for index, row in assets[:10].iterrows():
    trade_symbol = row['TSYMBOL']
    print(trade_symbol + " " + str(index))
    sess = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
    sess.mount('https://', TimeoutHTTPAdapter(max_retries=retries))
    
    url = 'https://finance.yahoo.com/quote/'+ trade_symbol + '/key-statistics'
    website = getdata(url, sess, 1)
    if not website:
        continue

    
    soup = bs(website.text, 'lxml')
    alldata = soup.find_all('tbody')

    try:
        table = alldata[2].find_all('tr')
    except:
        print('error 1 ' + trade_symbol + '*******************************')
        print(trade_symbol)
        continue

    try:
        table_td = table[2].find_all('td')
    except:
        print('error 2 ' + trade_symbol + '*******************************')
        print(trade_symbol)
        continue

    if "Shares Outstanding" in str(table_td[0].text):
        shrs = str(table_td[1].text)
        if 'B' in shrs:
            exp = 1000000000
        if 'T' in shrs:
            exp = 1000000000000
        if 'M' in shrs:
            exp = 1000000
        try:
            shrs = float(shrs.replace('B','').replace('M','').replace('T','')) * exp
        except:
            shrs = ''
            print('error 3 ' + trade_symbol + '*******************************')
            continue

    url = 'https://finance.yahoo.com/quote/'+ trade_symbol + '/history'
    website = getdata(url, sess, 1)
    if not website:
        continue
    soup = bs(website.text, 'lxml')
    alldata = soup.find_all('tbody')

    try:
        table = alldata[0].find_all('tr')
        
        prev = 1
        table_td = table[0].find_all('td')
        table_td_prev = table[prev].find_all('td')
        
        if(len(table_td)<5):
            table_td = table[1].find_all('td')
            prev = prev+1
        if(len(table_td_prev)<5):
            table_td_prev = table[prev+1].find_all('td')
        

        date = table_td[0].text
        date = datetime.strptime(date, '%b %d, %Y').strftime('%Y%m%d')
        prc = float(table_td[5].text)
        vol = str(table_td[6].text).replace(',','')
        vol = float(vol)
        ret = float(table_td[5].text)/float(table_td_prev[5].text)

    except:
        print('error 4 ' + trade_symbol + '*******************************')
        continue
    mktdata = mktdata.append({'asset_id': row['PERMNO'], 'date': date, 'prc': prc, 'vol': vol, 'ret': ret, 
                             'shrout': shrs, 'shrccd': row['SHRCD'], 'siccd': row['SICCD']}, ignore_index = True)


JJSF 0


KeyboardInterrupt: 

In [7]:
stkdb = pd.read_csv('data/STK_DB_INIT_2020.csv', index_col=0)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [8]:
stkdb

permno       caldt  shrcd   siccd        prc     vol       ret  \
0         10001.0  2007-01-03   11.0  4920.0  11.100000  4311.0  0.000000   
1         10001.0  2007-01-04   11.0  4920.0  11.360000  4600.0  0.023423   
2         10001.0  2007-01-05   11.0  4920.0  11.250000   510.0 -0.009683   
3         10001.0  2007-01-08   11.0  4920.0  11.345000     0.0  0.008444   
4         10001.0  2007-01-09   11.0  4920.0  11.240000   600.0 -0.009255   
...           ...         ...    ...     ...        ...     ...       ...   
13600676  82932.0  2020-10-29   11.0  2731.0  30.790001   200.0 -0.036910   
13600677  82932.0  2020-10-30   11.0  2731.0  30.790001     0.0  0.000000   
13600678  82932.0  2020-11-02   11.0  2731.0  31.719999   400.0  0.030205   
13600679  82932.0  2020-11-03   11.0  2731.0  32.139999   500.0  0.013241   
13600680  82932.0  2020-11-04   11.0  2731.0  32.139999     0.0  0.000000   

              shr     rf  
0          2959.0  0.022  
1          2959.0  0.022  
2          2959.0  0.022  
3          2959.0  0.022  
4          2959.0  0.022  
...           ...    ...  
13600676  55992.0    NaN  
13600677  55992.0    NaN  
13600678  55990.0    NaN  
13600679  56005.0    NaN  
13600680  56005.0    NaN  

[13599500 rows x 9 columns]